Import libraries

In [2]:
from predibase import Predibase, FinetuningConfig, DeploymentConfig, PredibaseClient
# from google.colab import drive
import os
import pandas as pd

predibase api key

In [3]:
pb = Predibase(api_token='pb_xBAMZ4FhUViAPJ_PppYx1Q')

Connected to Predibase as User(id=23cb6aeb-00c2-474c-8896-3036b8f920d2, username=Akhil)

path validation for dataset or anything

In [4]:
# file_path = '/Users/agenc/agencVenv/Ariel_Final.csv'

def file_path_validation(file_path):
    if os.path.isfile(file_path):
        print(f"Dataset found at {file_path}")
    else:
        raise FileNotFoundError(f"Dataset not found at {file_path}")

Upload abd get uploaded datasets from predibase

In [5]:
def upload_data(data_path, name):
    pb.datasets.from_file(data_path, name=name)
    return ('Uploaded!')

def get_data_from_predibase(name_of_the_dataset):
    return pb.datasets.get(name_of_the_dataset)

In [11]:
# if os.access(file_path, os.R_OK):
#     print("File is readable.")
# else:
#     print("File is not readable.")

Deployment config for predibase, required parameters (name, description).
Note - "t5-small" model does not work on predibase when you try to deploy it as they are not able to host the model as of now.

In [12]:
def deployment_config(name, description, base_model="mixtral-8x7b-instruct-v0-1", accelerator="a10_24gb_100",):
    deployment_config = DeploymentConfig(
        base_model=base_model,
        accelerator=accelerator,
    )

    deployment = pb.deployments.create(
        name=name,
        description=description,
        config=deployment_config
    )

Create adaptor on predibase using config = FinetuningConfig(). Required parameters = (dataset, repo_name, description) NOTE - Ignore the ssl cerrificate error after you run the create_adapter function.

In [7]:
def create_adapter(dataset, repo_name, description, base_model="mixtral-8x7b-instruct-v0-1", epochs=3, rank=3, learning_rate=0.0002):
    repo = pb.repos.create(name=repo_name, description=description, exists_ok=True)
    adapter = pb.adapters.create(
        config=FinetuningConfig(
            base_model=base_model,
            epochs=epochs,
            rank=rank,
            learning_rate=learning_rate
        ),
        dataset=dataset,
        repo=repo,
        description="changing epochs, rank, and learning rate"
    )
    return adapter

In [8]:
# client = pb.deployments.client("my-t5-small-deployment-test")
# result = client.generate(
#     "Generate a marketing campaign for Instagram with a friendly and casual tone for the summer season. Word limit: 100 words."
# )
# print(result)

In [13]:
# deployment_config('brainstorming-16-05-2024', 'brainstorming', base_model='mixtral-8x7b-instruct-v0-1')

In [14]:
create_adapter(dataset=get_data_from_predibase('Indian_Seasonalities'), repo_name='brainstorming', description='brainstorming-16-05-2024', base_model='mixtral-8x7b-instruct-v0-1', learning_rate=0.0001, epochs=120)

Successfully requested finetuning of mixtral-8x7b-instruct-v0-1 as `brainstorming/5`. (Job UUID: fe7717c1-29bf-4726-924c-96d1cf9a7c00).

Watching progress of finetuning job fe7717c1-29bf-4726-924c-96d1cf9a7c00. This call will block until the job has finished. Canceling or terminating this call will NOT cancel or terminate the job itself.

Job is starting. Total queue time: 0:00:46         
Waiting to receive training metrics...



SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)